<a href="https://colab.research.google.com/github/DrevnytskaYuliia/Data-Management-trying-to-become-someone-fr-/blob/main/B_PROJECT_DATA_MANAGEMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Motivation and Introduction**

The retail industry has changed due to the development of eCommerce platforms, which have left behind extensive digital records of customer behaviour. Every click, view, and purchase generates useful data that businesses can use to get better understanding of their clients, enhance user experience, and boost sales. However, there are analytical and technical difficulties in analysing such large amounts of behavioural data.

The user interaction data used in this project was gathered over a seven-month period from a large multi-category online store, recording over 285 million events such as product views, cart additions, and purchases. Our goal is to extract insights from this behavioural data so that we can make better decisions in areas like revenue forecasting, personalised marketing, and customer segmentation.

We use a mix of classification, regression, and clustering methods to investigate the possibilities of the data. These techniques allow us to predict the likelihood of transformation, identify different types of customers, and estimate future customer value.

**Dataset description**

The dataset used in this project is titled **“eCommerce behavior data from multi category store”**, publicly available on Kaggle.
It was collected as part of the Open CDP (Customer Data Platform) project.

It consists of approximately 67.5 million rows and captures user behavior events over a 7-month period (October 2019 to April 2020). Each row represents an interaction between a user and a product on a multi-category eCommerce platform.

**Total values:** 67501979

Main features of each variable in the dataset:


*   **event_time:** Records the exact timestamp of the user interaction with the platform (in UTC format).
*   **event_type:** Indicates the type of user action. Possible values include:

**view** – the user viewed a product

**cart** – the user added a product to cart

**remove_from_cart** – the user removed a product from cart

**purchase** – the user completed a purchase

*   **product_id:** A unique identifier for the product.
*   **category_id:** Numeric ID representing the product category.
*   **category_code:** Hierarchical string that describes the product category (e.g., electronics.smartphone, appliances.kitchen.)
*   **brand:** Name of the product brand.
*   **price:** Price of the product at the time of the interaction.
*   **user_id:** Unique anonymous identifier for the user.
*   **user_session:** Session ID used to group actions by browsing sessions.

**Research questions tackled**

**1. Clustering**

How can we group users based on their shopping behavior (e.g., frequency of views, cart additions, and purchases), and what distinct customer personas emerge from these clusters?

1.1 Can we identify distinct shopper personas (ex. window shoppers, cart abandoners, impulse buyers etc.)

1.2 How do these personas differ by product category preferences or average price points?

1.3 Are there differences in when and how often these groups shop?

1.4 Do patterns like repeat visits, short vs. long sessions, or quick purchase decisions help distinguish these clusters?

**2. Regression**

Can we predict the total revenue generated by a user based on their product browsing and cart behavior?

2.1  How does the number of product views and cart additions relate to the purchase amount?

2.2 Does the average price of viewed or carted products influence the final amount spent?

2.3 Do users who interact with multiple product categories tend to spend more?

2.4 Does recency or frequency of interaction affect total spending?

**3. Classification**

Can we classify whether a user will make a purchase within the next 7 days based on their recent browsing behavior?

3.1  Which behavioral signals are most predictive of an upcoming purchase?

3.2  Does cart behavior (e.g., added but not purchased) increase the likelihood of a purchase soon?

3.3 How well can we predict the likelihood of a purchase in the next 7 days?

3.4 Can we detect patterns in user behavior that help us intervene earlier?

**References**
**TO FIND**

**Import**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
!pip install kaggle

# **STEP 1: Load Data**


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# 2. Set the path to a specific CSV file from the dataset
file_path = "2019-Oct.csv"

# 3. Load into a DataFrame
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "mkechinov/ecommerce-behavior-data-from-multi-category-store",
    file_path
)

# 4. Preview
print("First 5 records:", df.head())

/tmp/ipython-input-3-1741159564.py:8: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [ ]:
#we have decided to go with sample of the data, understanding our large dataset limitations and also I added the index restart

df_sampled = (df.groupby('event_type', group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42)).reset_index(drop=True))

In [ ]:
print(df_sampled.head())

In [ ]:
#lets look if sampling is sucessfull. It is, we have got 10 percent, that follows the shape of original data

print("Original shape:", df.shape)
print("Sampled shape:", df_sampled.shape)

print("Original nulls:\n", df.isnull().sum())
print("Sampled nulls:\n", df_sampled.isnull().sum())

# STEP 2: Prepare and investigate data

In [ ]:
df_sampled.info()

In [ ]:
# then we need to change event_time to processable format

df_sampled['event_time'] = pd.to_datetime(df_sampled['event_time'], errors='coerce')

In [ ]:
#looking at data - sad sad
df_sampled.isnull().sum()

In [ ]:
# As we have a lot of missing vakues for category_code and brand, we have decided to fill them

df_sampled['brand'] = df_sampled['brand'].fillna('unknown')
df_sampled['category_code'] = df_sampled['category_code'].fillna('unknown')

In [ ]:
#sucessfully filled

df_sampled.isnull().sum()

In [ ]:
#looking for the duplicates

df_sampled.duplicated().sum()

In [ ]:
#it showed us exact duplicates, so it would be safe to delete those

df_sampled = df_sampled.drop_duplicates()

In [ ]:
#Also lets look at the prices and see if there is something unreasinable, so 0 or with minus sign

bad_prices = df_sampled[df_sampled['price'] <= 0]
print("Rows with price <= 0:", len(bad_prices))

bad_prices.head()

In [ ]:
#so we are dropping price that is 0 or smaller, but inly for cart and purchase. because view can help us with behavioural analysis still and does no harm for predicting revenue or values

df_sampled = df_sampled[~((df_sampled['price'] <= 0) & (df_sampled['event_type'].isin(['cart', 'purchase'])))]


In [ ]:
#also checked if there are unrealistically big prices

df_sampled.sort_values(by='price', ascending=False).head()

In [ ]:
#also we need to see if some of the users are suspicious, so looked at the

df_sampled['user_id'].value_counts().head(20)

In [ ]:
#looked for suspicious activity, but bro  is probably a window shoppper
top_user = df_sampled[df_sampled['user_id'] == 512475445]
top_user['event_type'].value_counts()

# Data preparation (User DF)

features Extract additional features чи це будні дні чи вихідні
Можна зробити лейбли


In [ ]:
#adding tags to see how many times each user vieweed, put something into cart or purchased by adding aditional columns

df_sampled['view'] = (df_sampled['event_type'] == 'view').astype(int)
df_sampled['cart'] = (df_sampled['event_type'] == 'cart').astype(int)
df_sampled['purchase'] = (df_sampled['event_type'] == 'purchase').astype(int)

In [ ]:
#doing separate columns for hours

df_sampled['hour_of_day'] = df_sampled['event_time'].dt.hour
df_sampled['weekday_name'] = df_sampled['event_time'].dt.day_name()

# Tag weekends using the weekday name
df_sampled['is_weekend'] = df_sampled['weekday_name'].isin(['Saturday', 'Sunday'])

In [ ]:
df_sampled.head()

In [ ]:
#then decided to concentrate on users and do a new df to define their actions grouping by user id. Also aggregate basic statistics per user

views = df_sampled[df_sampled['view'] == 1]
carts = df_sampled[df_sampled['cart'] == 1]
purchases = df_sampled[df_sampled['purchase'] == 1] #price-based stats filering

avg_price_viewed = views.groupby('user_id')['price'].mean().reset_index(name='avg_price_viewed') #separation by groups
avg_price_carted = carts.groupby('user_id')['price'].mean().reset_index(name='avg_price_carted')
total_spent = purchases.groupby('user_id')['price'].sum().reset_index(name='total_spent')

user_df = df_sampled.groupby('user_id').agg(
    total_views=('view', 'sum'),
    total_cart_additions=('cart', 'sum'),
    total_purchases=('purchase', 'sum'),
    num_categories=('category_code', pd.Series.nunique),
    num_products_viewed=('product_id', pd.Series.nunique),
    first_action=('event_time', 'min'),
    last_action=('event_time', 'max'),
    num_events=('event_time', 'count')
).reset_index() # Calculate base user metrics

# Merge it all
user_df = user_df.merge(avg_price_viewed, on='user_id', how='left')
user_df = user_df.merge(avg_price_carted, on='user_id', how='left')
user_df = user_df.merge(total_spent, on='user_id', how='left')

In [ ]:
#also fill avg_price_carted and total_spent with 0

user_df['avg_price_carted'] = user_df['avg_price_carted'].fillna(0)
user_df['total_spent'] = user_df['total_spent'].fillna(0)

In [ ]:
user_df['activity_days'] = (user_df['last_action'] - user_df['first_action']).dt.days + 1 #number of active days (added 0, to add the day of a start)

user_df['event_frequency'] = user_df['num_events'] / user_df['activity_days'] #event frequency (number per day)

user_df['cart_to_view_ratio'] = user_df['total_cart_additions'] / (user_df['total_views'] + 1e-5)

user_df['purchase_to_cart_ratio'] = user_df['total_purchases'] / (user_df['total_cart_additions'] + 1e-5)

user_df['purchase_to_view_ratio'] = user_df['total_purchases'] / (user_df['total_views'] + 1e-5)

user_df['cart_to_purchase_ratio'] = user_df['total_cart_additions'] / (user_df['total_purchases'] + 1e-5)


user_df['cart_to_purchase_ratio'] = user_df['cart_to_purchase_ratio'].fillna(0)
user_df['cart_to_view_ratio'] = user_df['cart_to_view_ratio'].fillna(0)
user_df['purchase_to_view_ratio'] = user_df['purchase_to_view_ratio'].fillna(0)
user_df['purchase_to_cart_ratio'] = user_df['purchase_to_cart_ratio'].fillna(0)

In [ ]:
# % of events on weekends
weekend_share = df_sampled.groupby('user_id')['is_weekend'].mean().reset_index(name='weekend_activity_ratio')

# Most active hour of the day (mode)
active_hour = df_sampled.groupby('user_id')['hour_of_day'].agg(lambda x: x.mode().iloc[0]).reset_index(name='most_active_hour')


In [ ]:
#Also wanna sort it to improve readability

user_df = user_df[
    [    'user_id',

        # Activity
        'total_views', 'total_cart_additions', 'total_purchases',
        'num_events', 'activity_days', 'active_hour' 'event_frequency', 'weekend_share'

        # Behavior
        'cart_to_view_ratio', 'cart_to_purchase_ratio', 'purchase_to_view_ratio',

        # Monetary
        'avg_price_viewed', 'avg_price_carted', 'total_spent',

        # Diversity of engagement
        'num_categories', 'num_products_viewed',

        # Timestamps
        'first_action', 'last_action' ]]

In [ ]:
user_df.head()

# STEP 3: Clustering

# STEP 4: Regression modeling

# STEP 5: Classification